In [2]:
import re
retorno = re.findall("^[0-9]{2}([^@]+)","952345SA#124@APNNE")
retorno

['2345SA#124']

In [1]:
def calculo_daora(x,y):
    if y == 0:
        return 0
    elif y == 1:
        return x
    else : 
        return x +calculo_daora(x,y-1)

valor = calculo_daora(2,500)
valor

1000